In [24]:
import time, sys, os, glob, re, multiprocessing, psutil, tqdm
from pathlib import Path
import numpy as np

import conll


In [2]:
conlltxt = """# newdoc id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600
# sent_id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600-0001
# newpar id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600-p0001
# text = Elena's motorcycle tour through the region around Chernobyl has revived interest in one of the most serious nuclear disasters in history.
1-2	Elena's	_	_	_	_	_	_	_	_
1	Elena	Elena	PROPN	NNP	Number=Sing	4	nmod:poss	4:nmod:poss	_
2	's	's	PART	POS	_	1	case	1:case	_
3	motorcycle	motorcycle	NOUN	NN	Number=Sing	4	compound	4:compound	_
4	tour	tour	NOUN	NN	Number=Sing	11	nsubj	11:nsubj	_
5	through	through	ADP	IN	_	7	case	7:case	_
6	the	the	DET	DT	Definite=Def|PronType=Art	7	det	7:det	_
7	region	region	NOUN	NN	Number=Sing	4	nmod	4:nmod:through	_
8	around	around	ADP	IN	_	9	case	9:case	_
9	Chernobyl	Chernobyl	PROPN	NNP	Number=Sing	7	nmod	7:nmod:around	_
10	has	have	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	11	aux	11:aux	_
11	revived	revive	VERB	VBN	Tense=Past|VerbForm=Part	0	root	0:root	_
12	interest	interest	NOUN	NN	Number=Sing	11	obj	11:obj	_
13	in	in	ADP	IN	_	14	case	14:case	_
14	one	one	NUM	CD	NumType=Card	12	nmod	12:nmod:in	_
15	of	of	ADP	IN	_	20	case	20:case	_
16	the	the	DET	DT	Definite=Def|PronType=Art	20	det	20:det	_
17	most	most	ADV	RBS	_	18	advmod	18:advmod	_
18	serious	serious	ADJ	JJ	Degree=Pos	20	amod	20:amod	_
19	nuclear	nuclear	ADJ	JJ	Degree=Pos	20	amod	20:amod	_
20	disasters	disaster	NOUN	NNS	Number=Plur	14	nmod	14:nmod:of	_
21	in	in	ADP	IN	_	22	case	22:case	_
22	history	history	NOUN	NN	Number=Sing	14	nmod	14:nmod:in	SpaceAfter=No
23	.	.	PUNCT	.	_	11	punct	11:punct	_
"""

In [3]:
tree=conll.conll2tree(conlltxt)

In [7]:
print(type(tree))
kk = tree[1]['gov'].keys()
print(kk)
for k in kk:
    print(k)
    
kl = list(kk)
print(kl)
print("\n",tree)

<class 'conll.Tree'>
dict_keys([4])
4
[4]

 # newdoc id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600
# newpar id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600-p0001
# sent_id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600-0001
# text = Elena's motorcycle tour through the region around Chernobyl has revived interest in one of the most serious nuclear disasters in history.
1	Elena	Elena	PROPN	NNP	Number=Sing	4	nmod:poss	4:poss	_
2	's	's	PART	POS	_	1	case	1:case	_
3	motorcycle	motorcycle	NOUN	NN	Number=Sing	4	compound	4:compound	_
4	tour	tour	NOUN	NN	Number=Sing	11	nsubj	11:nsubj	_
5	through	through	ADP	IN	_	7	case	7:case	_
6	the	the	DET	DT	Definite=Def|PronType=Art	7	det	7:det	_
7	region	region	NOUN	NN	Number=Sing	4	nmod	4:through	_
8	around	around	ADP	IN	_	9	case	9:case	_
9	Chernobyl	Chernobyl	PROPN	NNP	Number=Sing	7	nmod	7:around	_
10	has	have	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	11	aux	11:aux	_
11	revived	r

In [8]:
def headOfList(idlist,tree):
    if len(idlist):
        for i in idlist:
            for hd in tree[i]['gov'].keys():
                hdlist.append(hd)
    return hdlist

In [9]:
headlist = []
for i in range(1, len(tree)):
    for hd in tree[i]['gov'].keys():
        headlist.append(hd)    
print(headlist)

hhl = headOfList(range(1, len(tree)),tree)
print(hhl)

[4, 1, 4, 11, 7, 7, 4, 9, 7, 11, 0, 11, 14, 12, 20, 20, 18, 20, 20, 14, 22, 14]
[4, 1, 4, 11, 7, 7, 4, 9, 7, 11, 0, 11, 14, 12, 20, 20, 18, 20, 20, 14, 22, 14]


In [10]:
def haveCousin(hdset, tree):
    hdset.remove(0) #we are going to find head of headset while 0 haven't head
    #find head of headset, an id can appears more than once if there are multiple heads in one stage 
    hdofhdList = headOfList(hdset, tree)
    hdofhdSet = set(hdofhdList) #eliminate duplication
    return len(hdofhdList)!= len(set(hdofhdSet)), hdofhdSet

In [11]:
def treeHeight(tree):
    """
    find tree height with different head id. For each 'stage' of tree, we compte only one head. 
    if one node is a head, it means there is one more stage for its son (dependent). 
    while in one stage, maybe there are more than one head(who has branche), so we need delete the others
    """
    height = -1
    headset = set(headOfList(range(1, len(tree)),tree)) #all head id in current tree
    print("head id of current tree:", headset)
    
    branch = 0
    cousin = True #if there are cousins in headset
    while(cousin):
        cousin, headset = haveCousin(headset, tree) 
        print("headset == ", headset)
        branch += 1
        print(branch)
        
        if(not cousin):
            height = len(headset)+ branch
            
    return height

In [12]:
ht = treeHeight(tree)
print("tree height = ", ht)

head id of current tree: {0, 1, 4, 7, 9, 11, 12, 14, 18, 20, 22}
headset ==  {0, 4, 7, 11, 12, 14, 20}
1
headset ==  {0, 4, 11, 12, 14}
2
headset ==  {0, 11, 12}
3
headset ==  {0, 11}
4
tree height =  6


##### node height

In [37]:
def getHeadId(idx, tree): #put to private
    for hd in tree[idx]['gov'].keys():#hd is unique for each node
        return hd

In [38]:
#dep length
#for one node, find the depth by track back to the root

def depLength(nodeId, tree):
    length = 1 #every node have at least length 1 to the node 0
    head = getHeadId(nodeId, tree)
    while(head != 0):
        idx = head
        head = getHeadId(idx, tree)
        length += 1
    return length

In [39]:
assert(depLength(11,tree) == 1)
assert(depLength(12,tree) == 2)
assert(depLength(4,tree) == 2)
assert(depLength(1,tree) == 3)
assert(depLength(2,tree) == 4)
assert(depLength(21,tree) == 5)

Then stock deplength of nodes by tag of token for each language and find the average value 